###### TRYING RANDOM FOREST

In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.

In [1]:
# !aws configure

In [3]:
import mlflow
# S2 :- Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-205-141-180.compute-1.amazonaws.com:5000/")

In [4]:
# Set or Create an experiment
mlflow.set_experiment("EXP 5 - ML Algo with HP tuning")

<Experiment: artifact_location='s3://interview-mlflow-bucket/851239149715883194', creation_time=1732038163076, experiment_id='851239149715883194', last_update_time=1732038163076, lifecycle_stage='active', name='EXP 5 - ML Algo with HP tuning', tags={}>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df = pd.read_csv("/content/reddit_preprocessed.csv").dropna()
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [10]:
# S1 :- Remapping (Skipping it)

# S2 :- drop nan in category
df = df.dropna(subset=['category'])

# S3 :- set/create vectorizer
ngram_range = (1,3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# S4 :-
smote = SMOTE(random_state=42)
X_resampled,y_resampled = smote.fit_resample(X,y)

# S5 :- split the data
X_train,X_test ,y_train,y_test = train_test_split(X_resampled,y_resampled, test_size = 0.2,random_state = 42, stratify=y_resampled)

# S6 :- Func to log result in mlflow
def log_mlflow(model_name,model,X_trian,X_test,y_train,y_test):
  # log the model type
  mlflow.set_tag("mlflow.runName",f"{model_name}_SMOTE_TFIDF_Trigrams")
  mlflow.set_tag("experiment_type","alogorithm comparision")

  # log model name as a param
  mlflow.log_param("algo_name",model_name)

  # train model
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)

  # Log accuracy
  accuracy = accuracy_score(y_test,y_pred)
  mlflow.log_metric("accuracy",accuracy)

  # Log Classification report
  classification_rep = classification_report(y_test,y_pred,output_dict=True)
  for label,metrics in classification_rep.items():
    if isinstance(metrics,dict):
      for metric,value in metrics.items():
        mlflow.log_metric(f"{label}_{metric}",value)

  # log the model
  mlflow.sklearn.log_model(model,f"{model_name}_model")

# S6 :- optuna objective function for Random Forest
def objective_rf(trial):
  n_estimators = trial.suggest_int("n_estimators",50,300)
  max_depth = trial.suggest_int("max_depth",3,20)
  min_samples_split = trial.suggest_int("min_samples_split",2,20)
  min_samples_leaf = trial.suggest_int("min_samples_leaf",1,20)

  # RF setup
  model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split,random_state=42)

  return accuracy_score(y_test,model.fit(X_train,y_train).predict(X_test))

def run_optuna_experiment():
  study = optuna.create_study(direction="maximize")
  study.optimize(objective_rf,n_trials=5)

  # Get the best params & log the best model
  best_params = study.best_params
  best_model = RandomForestClassifier(n_estimators = best_params["n_estimators"],
                                      max_depth = best_params["max_depth"],
                                      min_samples_split = best_params["min_samples_split"],
                                      min_samples_leaf = best_params["min_samples_leaf"],
                                      random_state=42)
  log_mlflow("RandomForest",best_model,X_train,X_test,y_train,y_test)

# run experiment
run_optuna_experiment()

[I 2024-11-20 07:27:12,272] A new study created in memory with name: no-name-cfcf4b92-03af-4b31-bcc9-63ca981b799d
[I 2024-11-20 07:27:14,522] Trial 0 finished with value: 0.6495455506235468 and parameters: {'n_estimators': 79, 'max_depth': 8, 'min_samples_split': 14, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.6495455506235468.
[I 2024-11-20 07:27:31,804] Trial 1 finished with value: 0.7108433734939759 and parameters: {'n_estimators': 295, 'max_depth': 20, 'min_samples_split': 13, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.7108433734939759.
[I 2024-11-20 07:27:39,229] Trial 2 finished with value: 0.6959416613823716 and parameters: {'n_estimators': 190, 'max_depth': 17, 'min_samples_split': 8, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.7108433734939759.
[I 2024-11-20 07:27:41,614] Trial 3 finished with value: 0.6516592686535616 and parameters: {'n_estimators': 87, 'max_depth': 8, 'min_samples_split': 14, 'min_samples_leaf': 9}. Best is trial 1 with value: